# Pytorch: modules

In the previous notebook, we have laboriously trained a neural network written by hand. Pytorch actually contains many modules that make the definition of a network, the data collection and the training very easy.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Neural network module

The `torch.nn` module allows to easily define complicated networks. It implements basic linear and convolutional layers, and allows to build with them easily. To do so, we define a `nn.Module` class.

This class has two methods:

* `__init__` specifies the different layers of the network
* `forward` defines how to go from the input to the output using the layers defined in the init.

For instance, this is how you would define a simple neural network with one hidden layer of size $100$, that takes $10$ inputs and returns $3$ outputs:

In [ ]:
class Two_layers(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 100)
        self.layer2 = nn.Linear(100, 3)

    def forward(self, x):
        y = self.layer1(x)
        y = torch.tanh(y)
        y = self.layer2(y)
        return y 
    
net = Two_layers()

In [ ]:
x = torch.randn(1000, 10)
net(x)

# Optim module

The `torch.optim` module then allows to easily learn the parameters of the net with backprop. An optimizer, like SGD, takes the parameters of the network as input.

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(lr=1e-2, params=net.parameters())

We can then compute a (toy) loss, backpropagate, and use the `optimizer.step` method to make an update:

In [ ]:
optimizer.zero_grad()
y = net(x)
loss = y.sum()
loss.backward()
optimizer.step()

And that is it ! 

# Datasets
There are also some convenient tools to load datasets. For instance, we will work with MNIST:

In [ ]:
from torchvision import datasets, transforms

Let's download the MNIST dataset, the train and testing data, and apply a simple normalizing transform.

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)

We can then define a loader that will automatically load the images when we train / test

In [ ]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

We can then loop over these loaders to get the images:

In [ ]:
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    plt.imshow(x[0, 0])
    break

We are now ready to train a network on MNIST ! 

# Training on MNIST

**Exercise 1**: Define a one hidden neural network for MNIST, with a hidden layer of size $1000$.

In [ ]:
class Two_layers(nn.Module):
    def __init__(self):
        super().__init__()
        # Init: here you define the different subparts of the network

    def forward(self, x):
        # Forward: here you define the different operations on the network
        return x
    
net = Two_layers()

In [ ]:
for x, y in train_loader:
    print(net(x))
    break

**Exercise 2** : Define an SGD optimizer for this network

In [ ]:
optimizer = # Your code: define an optimizer with the network's parameters

**Exercise 3**: Perform one training epoch on the data, and compute the testing accuracy. Use the cross entropy as loss.

In [ ]:
from torch.nn.functional import cross_entropy

In [ ]:
for i, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    # Your code here: compute the predicted output of the network, then the loss.
    # Use backprop to compute the gradients, and perform a step of the optimizer.

    loss = # Your code here
    if i % 100 == 0:
        print(i, loss.item())

In [ ]:
good_preds = 0
for x, y in test_loader:
    pred = net(x)
    good_preds += (pred.argmax(axis=1) == y).sum().item()

print('Accuracy = %.1f %%' % (100 * good_preds / 10000))

You should get ~ 90% accuracy.

**Exercise 4**: Now, train a convolutionnal network, and try to get the best accuracy.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.pool = nn.MaxPool2d(2)
        self.layer1 = nn.Linear(1600, 100)
        self.layer2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = self.layer1(x)
        x = torch.tanh(x)
        x = self.layer2(x)
        return x
    
net = CNN()

for x, y in train_loader:
    print(net(x))
    break

In [ ]:
optimizer = optim.SGD(params=net.parameters(), lr=0.1)

In [ ]:
for i, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    pred = net(x)
    loss = cross_entropy(pred, y)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(i, loss.item())

In [ ]:
good_preds = 0
for x, y in test_loader:
    pred = net(x)
    good_preds += (pred.argmax(axis=1) == y).sum().item()

print('Accuracy = %.1f %%' % (100 * good_preds / 10000))

**Exercise 5**: Put this model on a GPU, and compare the training time.
To put a model to GPU, you can to `net.to('cuda')`. The data `x` and `y` should also be put on the GPU.